# Creating your own EmissionModel

To create your own `EmissionModel` all you need to do is define each indiviudal emission type and relate them. This operation will define a tree-like structure where each model links to at least one other model. 

In the sections below we will detail how to define each different type of spectra operation but there are a few arguments needed by an ``EmissionModel` regardless of the operation:

- A **label** which identifies both the model and will be used to label the resultant emission.
- A grid...
- A component to act on...


In [ ]:
from synthesizer.dust.attenuation import PowerLaw
from synthesizer.dust.emission import Blackbody
from synthesizer.emission_models import EmissionModel
from synthesizer.grid import Grid
from unyt import dimensionless, kelvin

# Get the grid which we'll need for extraction
grid_dir = "../../../tests/test_grid"
grid_name = "test_grid"
grid = Grid(grid_name, grid_dir=grid_dir)


## Defining an extraction

To define an extraction we simply need to pass a `Grid` to extract from and a spectra key to extract (with the option of providing an escape fraction).

In [ ]:
transmitted = EmissionModel(
    "transmitted", grid=grid, extract="transmitted", fesc=0.3
)

You'll notice this is similar to using the premade models detailed above but now we have freedom over the `label` too (the first argument).

## Defining a combination

To define a combination we simply pass the models we want combined to the `combine` keyword (along with a label).

In [ ]:
# Define models to combine
linecont = EmissionModel(
    "linecont",
    grid=grid,
    extract="linecont",
    mask_attr="log10ages",
    mask_thresh=7 * dimensionless,
    mask_op="<",
    fesc=0.7,
)
nebular_cont = EmissionModel(
    "nebular_continuum",
    grid=grid,
    extract="nebular_continuum",
    mask_attr="log10ages",
    mask_thresh=7 * dimensionless,
    mask_op="<",
)

# Define the combined model
nebular = EmissionModel("nebular", combine=(linecont, nebular_cont))

## Defining an attenuation

To define an attenuated spectra we need a dust curve, the spectra to apply the dust to, and an optical depth (once again along with a label).

One thing we haven't mentioned up until now is that `tau_v` and `fesc` can both have strings passed instead of numbers. When a string is passed the spectra generator method will extract the attribute of a component stated in the string and use those extracted values for `tau_v` or `fesc`.

In [ ]:
attenuated = EmissionModel(
    "attenuated",
    dust_curve=PowerLaw(slope=-1),
    apply_dust_to=nebular,
    tau_v="tau_v",
)

## Including a mask

A mask can be included in any step by passing `mask_attr`, `mask_thresh`, and `mask_op`.

In [ ]:
masked_transmitted = EmissionModel(
    "masked_transmitted",
    grid=grid,
    extract="transmitted",
    fesc="fesc",
    mask_attr="log10ages",
    mask_thresh=7 * dimensionless,
    mask_op="<",
)

But we aren't tied to having only a single mask on a step. If we want to add more masks we can use `add_mask` as we did on the premade models, these will be combined with an `and` at the point of spectra generation.

In [ ]:
masked_transmitted.add_mask("metallicities", 0.01 * dimensionless, "<")
masked_transmitted.add_mask("log10ages", 6 * dimensionless, ">")
print(masked_transmitted)

## Including related models

In the code below we'll reconstruct the `TotalEmission` model with the Charlot&Fall+2000 like attenuation operation explicitly to demonstrate constructing a complex model.

In [ ]:
# Define the extractions
transmitted = EmissionModel(
    "transmitted",
    grid=grid,
    extract="transmitted",
    fesc=0.3,
)
incident = EmissionModel("incident", grid=grid, extract="incident", fesc=0.0)
escaped = EmissionModel("escaped", grid=grid, extract="transmitted", fesc=0.7)
linecont = EmissionModel(
    "linecont",
    grid=grid,
    extract="linecont",
    mask_attr="log10ages",
    mask_thresh=7 * dimensionless,
    mask_op="<",
    fesc=0.7,
)
nebular_cont = EmissionModel(
    "nebular_continuum",
    grid=grid,
    extract="nebular_continuum",
    mask_attr="log10ages",
    mask_thresh=7 * dimensionless,
    mask_op="<",
)

# Combine the extractions
nebular = EmissionModel("nebular", combine=(linecont, nebular_cont))
reprocessed = EmissionModel("reprocessed", combine=(nebular, transmitted))

# Apply the young and old dust attenuation
young_attenuated = EmissionModel(
    "young_attenuated",
    dust_curve=PowerLaw,
    apply_dust_to=reprocessed,
    tau_v=("tau_v", 0.67),
    mask_attr="log10ages",
    mask_thresh=7 * dimensionless,
    mask_op="<",
)
old_attenuated = EmissionModel(
    "old_attenuated",
    dust_curve=PowerLaw,
    apply_dust_to=reprocessed,
    tau_v="tau_v",
    mask_attr="log10ages",
    mask_thresh=7 * dimensionless,
    mask_op=">=",
)

# And combine them into a single attenuated spectra
attenuated = EmissionModel(
    "attenuated", combine=(young_attenuated, old_attenuated)
)
emergent = EmissionModel("emergent", combine=(attenuated, escaped))

# Create a dust emission model
dust_emission = EmissionModel(
    "dust_emission",
    generator=Blackbody(temperature=100 * kelvin),
    lum_intrinsic_model=incident,
    lum_attenuated_model=emergent["attenuated"],
)

# And bring everything together into the total emission
total = EmissionModel("total", combine=(emergent, dust_emission))

total.plot_emission_tree()
print(total)